In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import os
import numpy as np
import pyspark.sql.functions as f
import subprocess
from pyspark.sql.functions import min, max, col
from pyspark.sql.functions import *

In [2]:
sc = SparkContext()
sqlcontext = SQLContext(sc)

In [3]:
dir_in = "/user/hadoop/data/population_raw/local_dong"
args = "hdfs dfs -ls "+dir_in+" | awk '{print $8}'"
proc = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

s_output, s_err = proc.communicate()
all_dart_dirs = s_output.decode('utf-8').split()
print(all_dart_dirs)

['/user/hadoop/data/population_raw/local_dong/LOCAL_PEOPLE_DONG_201701.csv', '/user/hadoop/data/population_raw/local_dong/LOCAL_PEOPLE_DONG_201702.csv', '/user/hadoop/data/population_raw/local_dong/LOCAL_PEOPLE_DONG_201703.csv', '/user/hadoop/data/population_raw/local_dong/LOCAL_PEOPLE_DONG_201704.csv', '/user/hadoop/data/population_raw/local_dong/LOCAL_PEOPLE_DONG_201705.csv', '/user/hadoop/data/population_raw/local_dong/LOCAL_PEOPLE_DONG_201706.csv', '/user/hadoop/data/population_raw/local_dong/LOCAL_PEOPLE_DONG_201707.csv', '/user/hadoop/data/population_raw/local_dong/LOCAL_PEOPLE_DONG_201708.csv', '/user/hadoop/data/population_raw/local_dong/LOCAL_PEOPLE_DONG_201709.csv', '/user/hadoop/data/population_raw/local_dong/LOCAL_PEOPLE_DONG_201710.csv', '/user/hadoop/data/population_raw/local_dong/LOCAL_PEOPLE_DONG_201711.csv', '/user/hadoop/data/population_raw/local_dong/LOCAL_PEOPLE_DONG_201712.csv', '/user/hadoop/data/population_raw/local_dong/LOCAL_PEOPLE_DONG_201801.csv', '/user/hado

In [4]:
df = sqlcontext.read.format("com.databricks.spark.csv")\
    .option("header", "true").option("inferSchema", "true").load(all_dart_dirs)

In [16]:
df.count()

8456256

In [5]:
dong_df = sqlcontext.read.option("header","true").csv("data/cur_dongCode.csv")
dong_df.show()

+--------+--------+---+----+-----------+
|통계청행정동코드|행자부행정동코드|시도명|시군구명|       행정동명|
+--------+--------+---+----+-----------+
| 1101053|11110530| 서울| 종로구|        사직동|
| 1101054|11110540| 서울| 종로구|        삼청동|
| 1101055|11110550| 서울| 종로구|        부암동|
| 1101056|11110560| 서울| 종로구|        평창동|
| 1101057|11110570| 서울| 종로구|        무악동|
| 1101058|11110580| 서울| 종로구|        교남동|
| 1101060|11110600| 서울| 종로구|        가회동|
| 1101061|11110615| 서울| 종로구|종로1.2.3.4가동|
| 1101063|11110630| 서울| 종로구|    종로5.6가동|
| 1101064|11110640| 서울| 종로구|        이화동|
| 1101067|11110670| 서울| 종로구|       창신1동|
| 1101068|11110680| 서울| 종로구|       창신2동|
| 1101069|11110690| 서울| 종로구|       창신3동|
| 1101070|11110700| 서울| 종로구|       숭인1동|
| 1101071|11110710| 서울| 종로구|       숭인2동|
| 1101072|11110515| 서울| 종로구|      청운효자동|
| 1101073|11110650| 서울| 종로구|        혜화동|
| 1102052|11140520| 서울|  중구|        소공동|
| 1102054|11140540| 서울|  중구|        회현동|
| 1102055|11140550| 서울|  중구|         명동|
+--------+--------+---+----+-----------+
only showing top

In [6]:
import time

In [12]:
start = time.time()  # 시작 시간 저장
 

joined = df.join(dong_df, df.행정동코드 == dong_df.행자부행정동코드, how='right')
joined.show()
 
 
print("time :", time.time() - start) 

+--------+-----+--------+----------+-------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+------------+-------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+------------+--------+--------+---+----+----+
|   기준일ID|시간대구분|   행정동코드|    총생활인구수|남자0세부터9세생활인구수|남자10세부터14세생활인구수|남자15세부터19세생활인구수|남자20세부터24세생활인구수|남자25세부터29세생활인구수|남자30세부터34세생활인구수|남자35세부터39세생활인구수|남자40세부터44세생활인구수|남자45세부터49세생활인구수|남자50세부터54세생활인구수|남자55세부터59세생활인구수|남자60세부터64세생활인구수|남자65세부터69세생활인구수|남자70세이상생활인구수|여자0세부터9세생활인구수|여자10세부터14세생활인구수|여자15세부터19세생활인구수|여자20세부터24세생활인구수|여자25세부터29세생활인구수|여자30세부터34세생활인구수|여자35세부터39세생활인구수|여자40세부터44세생활인구수|여자45세부터49세생활인구수|여자50세부터54세생활인구수|여자55세부터59세생활인구수|여자60세부터64세생활인구수|여자65세부터69세생활인구수|여자70세이상생활인구수|통계청행정동코드|행자부행정동코드|시

In [8]:
from pyspark.sql.functions import broadcast

In [17]:
start = time.time()  # 시작 시간 저장

test = df.join(broadcast(dong_df), df.행정동코드 == dong_df.행자부행정동코드)
test.show()

print("time :", time.time() - start) 

+--------+-----+--------+----------+-------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+------------+-------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+------------+--------+--------+---+----+-----+
|   기준일ID|시간대구분|   행정동코드|    총생활인구수|남자0세부터9세생활인구수|남자10세부터14세생활인구수|남자15세부터19세생활인구수|남자20세부터24세생활인구수|남자25세부터29세생활인구수|남자30세부터34세생활인구수|남자35세부터39세생활인구수|남자40세부터44세생활인구수|남자45세부터49세생활인구수|남자50세부터54세생활인구수|남자55세부터59세생활인구수|남자60세부터64세생활인구수|남자65세부터69세생활인구수|남자70세이상생활인구수|여자0세부터9세생활인구수|여자10세부터14세생활인구수|여자15세부터19세생활인구수|여자20세부터24세생활인구수|여자25세부터29세생활인구수|여자30세부터34세생활인구수|여자35세부터39세생활인구수|여자40세부터44세생활인구수|여자45세부터49세생활인구수|여자50세부터54세생활인구수|여자55세부터59세생활인구수|여자60세부터64세생활인구수|여자65세부터69세생활인구수|여자70세이상생활인구수|통계청행정동코드|행자부행정동코드|